In [1]:
import tensorflow as tf
from sklearn.datasets import make_moons
import numpy as np

In [2]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
log_dir = "tf_logs/run-{}".format(now)

In [3]:
def fetch_data(step , batch_size , n_batches , Xm , ym , perms):
    
    starting = step * batch_size
    indicies = perms[ starting:starting+batch_size]
    return Xm[indicies , :] , ym[indicies].reshape(-1,1)

In [4]:

def logistic_regression(Xm , ym , learning_rate = 0.1):
    
    n_features = Xm.shape[1] 
    n_batches = 10
    batch_size = 10
    n_epoch = 10
    
    with tf.name_scope("Inputs"):
        X = tf.placeholder(tf.float32 , shape=(None , n_features) , name="X")
        y = tf.placeholder(tf.float32 , shape=(None , 1) , name="y")
    
    with tf.variable_scope("Weights" , reuse=tf.AUTO_REUSE) :
        W = tf.get_variable(name="W" , initializer=tf.random_uniform(shape=(n_features , 1) , minval=0.0 , maxval=1.0) , dtype=tf.float32 )
        b = tf.get_variable(name="b" , initializer=0.0, dtype=tf.float32 )
    
    with tf.variable_scope("HyperParameters" , reuse=tf.AUTO_REUSE):
        lr = tf.get_variable(name="lr" , initializer=learning_rate  , dtype=tf.float32)
    
    z = tf.add(tf.matmul(X , W) , b)
    prediction = tf.sigmoid(z , name="prediction")
    loss =  tf.reduce_sum(- y * tf.log(prediction) - (1 - y) * tf.log(1 - prediction) )
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
    training_op = optimizer.minimize(loss)
    
    init = tf.global_variables_initializer()
    
    loss_summary = tf.summary.scalar("Loss" , loss)
    file_writer  = tf.summary.FileWriter(logdir=log_dir)
    
    saver = tf.train.Saver()
    
    perms = np.random.permutation(Xm.shape[0])
    with tf.Session() as sess :
        sess.run(init)
        
        for epoch in range(n_epoch):
            for step in range(n_batches):
                Xbatch , ybatch = fetch_data(step , batch_size , n_batches , Xm , ym , perms)
                if step % 5 == 0 :
                    s = loss_summary.eval(feed_dict={X:Xm , y:ym})
                    file_writer.add_summary(s , step + epoch * n_batches)
                sess.run(training_op , feed_dict={X:Xbatch , y:ybatch})
        best_W , best_b = sess.run([W , b])
        saver.save(sess , "/tmp/model_final.ckpt")
        file_writer.close()


In [5]:
data = make_moons(n_samples=200 , noise=0.2)

Xm , ym = data
ym = ym.reshape(-1,1)

In [8]:
logistic_regression(Xm , ym , 1.0)